In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

# Set a new working directory
new_path = '../'
os.chdir(new_path)

In [ ]:
from morphomics import protocols, utils
import tomli

# Path to the toml file. Contains the parameters for each protocol
parameters_filepath = "examples/Parameter_files/Morphomics.Parameters_trial.toml"

# read the toml file
with open(parameters_filepath, mode="rb") as _parameter_file:
    parameters = tomli.load(_parameter_file)

# get the protocol
# from already saved protocol
# or initialize a new protocol with parameters
if parameters["load_previous_instance"]:
    last_instance_path = os.path.join(parameters["path_to_last_instance"], f'last_instance_{parameters["Parameters_ID"]}')
    protocol = utils.load_obj(last_instance_path)
else:
    protocol = protocols.Protocols(parameters, parameters["Parameters_ID"])

script_sequence = parameters["Protocols"]
print(script_sequence)

In [ ]:
# input
perform_input = getattr(protocol, 'Input')
perform_input()

In [ ]:
tmd_microglia = protocol.morphoframe['TMD']
print(tmd_microglia.shape)

In [ ]:
from tmd.view import plot
sample = tmd_microglia['barcodes'][0]
plot.barcode(sample)

In [ ]:
# clean frame
perform_input = getattr(protocol, 'Clean_frame')
perform_input()

In [ ]:
# load cleaned input

parameters['Protocols'].append('Load_data')
parameters['Load_data'] = {'folderpath_to_data': 'examples/trial_folder/tmd_cleaned/Morphomics.PID_trial.Cleaned',
                           'morphoframe_name': 'TMD'}# 
perform_input = getattr(protocol, 'Load_data')
perform_input()

In [ ]:
df = protocol.morphoframe['TMD']

In [ ]:
protocol.morphoframe.keys()

In [ ]:
parameters['Protocols'].append('Vectorizations')
parameters['Vectorizations'] = {'morphoframe_filepath' : 0,
                                'morphoframe_name' : "TMD",
                                'vect_method_parameters': {#'persistence_image': { "rescale_lims" : False,
                                #                                                     "xlims" : None,
                                #                                                     "ylims" : None,
                                #                                                     "bw_method" : 0.5,
                                #                                                     "barcode_weight" : None,
                                #                                                     "norm_method" : "sum",
                                #                                                     "resolution" : 20
                                #                                                 },
                                                            'lifespan_curve': { "rescale_lims" : False,
                                                                                    "xlims" : None,
                                                                                    "norm_method" : "id",
                                                                                    "resolution" : 50
                                                                                },
                                                            'betti_curve': { "rescale_lims" : False,
                                                                                    "xlims" : None,
                                                                                    "norm_method" : "id",
                                                                                    "resolution" : 50
                                                                                },
                                                            'life_entropy_curve' : { "rescale_lims" : False,
                                                                                    "xlims" : None,
                                                                                    "norm_method" : "id",
                                                                                    "resolution" : 50
                                                                                }

                                                         },
                                'save_data' : True,
                                'save_folder' : "examples/trial_folder/vectors",
                                'file_prefix' : 0
                                }
perform_vectorization = getattr(protocol, 'Vectorizations')
perform_vectorization()

In [ ]:
parameters['Protocols'].append('Embedding')
parameters['Embedding'] = {'morphoframe_filepath' : 0,
                           'morphoframe_name' : "TMD",
                           'vectors_to_embed' : 'lsc_bc_lec',
                           'filter_pixels' : False,
                           'normalize' : True,
                            'embed_method_parameters' : { 'pca' : {'n_components' : 3,
                                                                   'svd_solver' : True,
                                                                  'pca_version' : 'normal',
                                                                   },
                                                        
                                                    #    'umap' : {'n_components' : 3,
                                                     #             'n_neighbors' : 20,
                                                      #            'min_dist': 0.1,
                                                       #           'spread' : 3.,
                                                        #          'random_state' : 10,
                                                         #         'metric' : "manhattan",
                                                          #        'densmap' : False,
                                                           #       }
                               
                                                        },
                            'save_data' : True,
                            'save_folder' : "examples/trial_folder/embeddings",
                            'file_prefix' : 0
                           }
perform_embedding = getattr(protocol, 'Embedding')
perform_embedding()

In [ ]:
parameters['Protocols'].append('Save_EmbedInfo')
parameters['Save_EmbedInfo'] = {'morphoframe_filepath' : 0,
                                       'morphoframe_name' : 'TMD',
                                       'coordinate_key' : "pca",
                                       'coordinate_axisnames' : "pca_2",
                                       'condition_to_save' : ["Region",
                                                                "Model",
                                                                "Sex",
                                                            ],
                                       'save_folder' : "examples/trial_folder/reduction",
                                       'save_data' : True,
                                       'file_prefix' : 0 
                                        }
perform_embedding = getattr(protocol, 'Save_EmbedInfo')
perform_embedding()

In [ ]:
parameters['Protocols'].append('Save_EmbedInfo')
parameters['Save_EmbedInfo'] = {'morphoframe_filepath' : 0,
                                       'morphoframe_name' : "TMD",
                                       'fitted_embedder_filepath' : 0,
                                       'embed_method' : "pca",
                                       'save_folder' : "examples/trial_folder/reduction",
                                       'save_data' : True,
                                       'file_prefix' : 0 
                                        }
perform_embedding = getattr(protocol, 'Save_EmbedInfo')
perform_embedding()

In [ ]:
protocol.morphoframe['TMD'].Model

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

df = protocol.morphoframe['TMD']

# Mapping colors to models
color_map = {'A': 'red', 'B': 'blue', 'C': 'green', 'D': 'orange'}
df['color'] = df['Model'].map(color_map)

# Unpacking 3D vectors
df[['x', 'y', 'z']] = pd.DataFrame(df['pca'].tolist(), index=df.index)

# Plotting with Plotly
fig = px.scatter_3d(df, x='x', y='y', z='z', color='Model', color_discrete_map=color_map,
                    opacity=1, size_max=10)
fig.update_traces(marker=dict(size=3))
# Update layout for better viewing
fig.update_layout(
    title='3D Scatter Plot with Color by Model',
    scene=dict(
        xaxis=dict(title='PCA1'),
        yaxis=dict(title='PCA2'),
        zaxis=dict(title='PCA3')
    )
)

fig.show()

In [ ]:
# bootstrap
parameters["Bootstrap"]["morphoframe_name"] = "TMD"
parameters["Bootstrap"]["feature_to_bootstrap"] = ["barcodes", "bars"] 
perform_input = getattr(protocol, 'Bootstrap')
perform_input()

In [ ]:
barcode_example = protocol.morphoframe['bootstrap_frame']['barcodes'][0]
plot.barcode(barcode_example)

In [ ]:
protocol.morphoframe["TMD"].keys()

In [ ]:
parameters["Prepare_ReductionInfo"]["coordinate_key"] = ''
perform_input = getattr(protocol, 'Prepare_ReductionInfo')
perform_input()